In [24]:
# make connection to google drive to access data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# import relevant libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

%matplotlib inline

In [26]:
# set up directory paths to image files
combined_path_train = "/content/drive/My Drive/Deep Learning and Generative AI/C-NMC_Leukemia/combined_images_train"
#test_path = "/content/drive/My Drive/Deep Learning and Generative AI/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data"
combined_path_val = "/content/drive/My Drive/Deep Learning and Generative AI/C-NMC_Leukemia/combined_images_val"

In [27]:
import sys
sys.path.append('/content/keras-inceptionV4')

In [28]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, ResNet50, EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, ResNet50, EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [29]:
!git clone https://github.com/kentsommer/keras-inceptionV4.git

fatal: destination path 'keras-inceptionV4' already exists and is not an empty directory.


In [30]:
import sys
sys.path.append('/content/keras-inceptionV4')

In [31]:
from inception_v4 import create_model

# Create the model
model = create_model(num_classes=2, dropout_prob=0.2, weights='imagenet', include_top=True)

In [32]:
early_stop = EarlyStopping(monitor='val_loss',patience=3,verbose=1)

In [33]:
def build_model(base_model):
    base_model.trainable = False # freeze pre-trained layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x) # change to softmax for multi-class
    return Model(inputs=base_model.input, outputs=x)

In [34]:
# define image size (InceptionV3 expects at least 75x75)
img_size = (299, 299)
batch_size = 32

In [35]:
def preprocess_image(x):
    x = tf.image.resize(x, [299, 299])  # Resize
    if x.shape[-1] == 1:  # If grayscale, convert to RGB
        x = tf.image.grayscale_to_rgb(x)
    return x

# set up generator for image manipulation to add diversity in dataset
image_gen = ImageDataGenerator(rotation_range=20, # rotate 20 degrees
                               width_shift_range=0.10, # shift width by max of 5%
                               height_shift_range=0.10, # shift height by max of 5%
                               rescale=1./255, # rescale image by normalzing
                               shear_range=0.1, # cutting part of image (max 10%)
                               zoom_range=0.1, # zoom by 10% max
                               horizontal_flip=True, # horizontal flip
                               fill_mode='nearest', # fill missing pixels with nearest filled value
                               preprocessing_function=preprocess_image # preprocess image
                              )

In [36]:
# generator to get training images from combined path
train_image_gen = image_gen.flow_from_directory(combined_path_train,
                                                target_size=(299, 299),
                                                color_mode='rgb',
                                                batch_size=batch_size,
                                                class_mode='binary')

Found 4794 images belonging to 2 classes.


In [37]:
# generator to get validation images from combined path
val_image_gen = image_gen.flow_from_directory(combined_path_val,
                                               target_size=(299, 299),
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=True)

Found 2438 images belonging to 2 classes.


In [38]:
# load each model
models = {
    "InceptionV4": build_model(create_model(num_classes=2, dropout_prob=0.2, weights='imagenet', include_top=False)),
    "InceptionV3": build_model(InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size[0],img_size[1],3))),
    "ResNet50": build_model(ResNet50(weights='imagenet', include_top=False, input_shape=(img_size[0],img_size[1],3))),
    "EfficientNetB3": build_model(EfficientNetB3(weights='imagenet', include_top=False, input_shape=(img_size[0],img_size[1],3)))
}

In [39]:
# suppress any warnings
import warnings
warnings.filterwarnings('ignore')

In [40]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [41]:
# dictionary to store model performance
results = []

# train each model and store results
for name, model in models.items():
    print(f"Training {name}...")

    # compile model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    # train model
    history = model.fit(train_image_gen, validation_data=val_image_gen, epochs=10, callbacks=[early_stop], verbose=1)

    # evaluate on validation set
    y_true = val_image_gen.classes # true labels
    y_pred_prob = model.predict(val_image_gen) # predicted probabilities
    y_pred = (y_pred_prob > 0.4).astype(int) # convert to binary labels

    # calculate metrics
    accuracy = accuracy_score(y_true,y_pred)
    precision = precision_score(y_true,y_pred)
    recall = recall_score(y_true,y_pred)
    f1 = f1_score(y_true,y_pred)
    roc_auc = roc_auc_score(y_true,y_pred_prob)

    # store results
    results.append([name, accuracy, precision, recall, f1, roc_auc])

    print("\n")

Training InceptionV4...
Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 516s 3s/step - accuracy: 0.5019 - loss: 0.6820 - val_accuracy: 0.5000 - val_loss: 0.6639
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 247s 2s/step - accuracy: 0.5060 - loss: 0.6659 - val_accuracy: 0.5242 - val_loss: 0.6541
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.5401 - loss: 0.6486 - val_accuracy: 0.5016 - val_loss: 0.6338
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 237s 2s/step - accuracy: 0.5649 - loss: 0.6353 - val_accuracy: 0.7555 - val_loss: 0.6184
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 234s 2s/step - accuracy: 0.6331 - loss: 0.6176 - val_accuracy: 0.7683 - val_loss: 0.6006
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 235s 2s/step - accuracy: 0.6784 - loss: 0.6045 - val_accuracy: 0.7830 - val_loss: 0.5801
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 237s 2s/step - accuracy: 0.7165 - loss: 0.5816 - val_accuracy: 0.7523 - val_loss: 0.5655
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 236s 2s/step - accuracy: 0.7421 -

In [45]:
# convert results to DataFrame and display
df_results = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score", "AUC-ROC"])

In [ ]:
display(df_results)

In [50]:
# dictionary to store model performance
results2 = []

# train each model and store results
for name, model in models.items():
    print(f"Training {name}...")

    # evaluate on validation set
    y_true = train_image_gen.classes # true labels
    y_pred_prob = model.predict(train_image_gen) # predicted probabilities
    y_pred = (y_pred_prob > 0.4).astype(int) # convert to binary labels

    # calculate metrics
    accuracy = accuracy_score(y_true,y_pred)
    precision = precision_score(y_true,y_pred)
    recall = recall_score(y_true,y_pred)
    f1 = f1_score(y_true,y_pred)
    roc_auc = roc_auc_score(y_true,y_pred_prob)

    # store results
    results2.append([name, accuracy, precision, recall, f1, roc_auc])

    print("\n")

Training InceptionV4...
150/150 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step


Training InceptionV3...
150/150 ━━━━━━━━━━━━━━━━━━━━ 137s 917ms/step




In [ ]:
# convert results to DataFrame and display
df_results2 = pd.DataFrame(results2, columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score", "AUC-ROC"])

display(df_results2)